In [110]:
import requests
import datetime

def populate_list(number_days_window):
    populated_list = []
    all_days = requests.get('https://api.covid19api.com/dayone/country/israel/status/confirmed/live')
    #all_days = requests.get('https://api.covid19api.com/dayone/country/argentina/status/confirmed/live', verify=False)
    first=True
    for day in all_days.json():
        date_time_obj = datetime.datetime.strptime(day['Date'][:10], '%Y-%m-%d')
        if first:
            ordinal_first = date_time_obj.toordinal()
        n = date_time_obj.toordinal() - ordinal_first
        delta = 0
        if n>number_days_window:
            delta = day['Cases'] - populated_list[n-number_days_window][1] 
        first=False
        populated_list.append((n,day['Cases'], delta, date_time_obj))
    return populated_list

In [111]:
import pandas as pd
import numpy as np


def process_means(number_days_window, jump_days, total_lapse_day):
    my_populated_list = populate_list(number_days_window)
    means_list = []

    df = pd.DataFrame (my_populated_list,columns=['day-number','cases-count','delta','date'])
    
    for n in range(total_lapse_day,0,-jump_days):
        try:
            mean = round(df[-(n):-(n-(jump_days))]['delta'].mean())
        except:
            mean = round(df[-(n):-(n-(jump_days-1))]['delta'].mean())
        means_list.append(mean)
    return means_list

a = range(5)

print(process_means(30,14,70))

b = process_means(30,14,70)
from scipy.stats import linregress
print(linregress(a, b))

print(process_means(14,14,70))

b = process_means(14,14,70)
from scipy.stats import linregress
print(linregress(a, b))

print(process_means(10,14,70))

b = process_means(10,14,70)
from scipy.stats import linregress
print(linregress(a, b))

print(process_means(7,14,70))

b = process_means(7,14,70)
from scipy.stats import linregress
print(linregress(a, b))

print(process_means(5,14,70))

b = process_means(5,14,70)
from scipy.stats import linregress
print(linregress(a, b))

[88856, 33061, 21106, 27822, 48792]
LinregressResult(slope=-8536.699999999999, intercept=61000.8, rvalue=-0.4978262373274369, pvalue=0.3934008687256611, stderr=8586.364807647064)
[18476, 9304, 10166, 16357, 31143]
LinregressResult(slope=3238.7000000000003, intercept=10611.8, rvalue=0.5830919417892946, pvalue=0.3021139052493986, stderr=2605.2311855188586)
[10783, 6443, 7532, 12624, 24574]
LinregressResult(slope=3376.3, intercept=5638.6, rvalue=0.73667430662305, pvalue=0.15564257317915853, stderr=1789.4134038840773)
[6583, 4477, 5459, 9421, 18406]
LinregressResult(slope=2859.0, intercept=3151.2000000000007, rvalue=0.8009860526918378, pvalue=0.10333557615081423, stderr=1233.7452843543788)
[4344, 3214, 3998, 7016, 13949]
LinregressResult(slope=2301.2000000000003, intercept=1901.7999999999993, rvalue=0.8268293528688193, pvalue=0.08422280251798327, stderr=903.7826139804489)


In [4]:
from random import choice
from experta import *


class Light(Fact):
    """Info about the traffic light."""
    pass


class RobotCrossStreet(KnowledgeEngine):
    @Rule(Light(color='green'))
    def green_light(self):
        print("Walk")

    @Rule(Light(color='red'))
    def red_light(self):
        print("Don't walk")

    @Rule(AS.light << Light(color=L('yellow') | L('blinking-yellow')))
    def cautious(self, light):
        print("Be cautious because light is", light["color"])


engine1 = RobotCrossStreet()
engine1.reset()
engine1.declare(Light(color=choice(['green', 'yellow', 'blinking-yellow', 'red'])))
engine1.run()


Be cautious because light is blinking-yellow
